## 인스턴스 생성 조절에 메타클래스 사용

* 메타 클래스: 클래스를 어떻게 만드는 지 기술하는 클래스
* type을 상속받아 쓴다

In [ ]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")

class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')

if __name__ == '__main__':
    try:
        s = Spam()
    except TypeError as e:
        print(e)

    Spam.grok(42)

Can't instantiate directly
Spam.grok


* 객체를 하나만 만들어서 유지하고 싶은 경우

In [ ]:
class Singleton(type):
    def __init__(self, *args, **kwargs): #Spam 클래스가 생성되는 시점에 호출되는 메타 클래스의 메소드
        self.__instance = None
        super().__init__(*args, **kwargs)
        print("Singleton initiated")

    def __call__(self, *args, **kwargs): #Spam() 시에 호출되는 메소드
        print("Singleton called")
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs) # 왜 super()가 ??
            return self.__instance
        else:
            return self.__instance

class Spam(metaclass=Singleton):
    #__instance = None; #line 3
    def __init__(self):
        print('Creating Spam')

if __name__ == '__main__':
    a = Spam()
    b = Spam()
    print(Spam.__dict__)
    print(a)
    print(b)

Singleton initiated
Singleton called
Creating Spam
Singleton called
{'__module__': '__main__', '__init__': <function Spam.__init__ at 0x00000266FBE7AAF0>, '__dict__': <attribute '__dict__' of 'Spam' objects>, '__weakref__': <attribute '__weakref__' of 'Spam' objects>, '__doc__': None, '_Singleton__instance': <__main__.Spam object at 0x00000266FBE45C40>}


In [ ]:
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()

    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj
        
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name

if __name__ == '__main__':
    a = Spam('foo')
    b = Spam('bar')
    print('a is b:', a is b)
    c = Spam('foo')
    print('a is c:', a is c)

Creating Spam('foo')
Creating Spam('bar')
a is b: False
a is c: True


## 클래스 속성 정의 순서 수집

* prepare -> new -> init -> call 
* init 보다 먼저 뭘 하고 싶은게 있을때, new를 정의해서 사용한다.

In [ ]:
from collections import OrderedDict

class Typed:
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name

    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' +str(self._expected_type))
        instance.__dict__[self._name] = value

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str

class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)

    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()

class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)

class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price

if __name__ == '__main__':
    s = Stock('GOOG',100,490.1)
    print(s.name)
    print(s.as_csv())
    try:
        t = Stock('AAPL','a lot', 610.23)
    except TypeError as e:
        print(e)



GOOG
GOOG,100,490.1
Expected <class 'int'>


## 옵션 매개변수를 받는 메타클래스 정의

In [4]:
class MyMeta(type):
    @classmethod
    def __prepare__(cls, name, bases, *, debug=False, synchronize=False):
        return super().__prepare__(name, bases)

    def __new__(cls, name, bases, ns, *, debug=False, synchronize=False):
        return super().__new__(cls, name, bases, ns)
        
    def __init__(self, name, bases, ns, *, debug=False, synchronize=False):
        super().__init__(name, bases, ns)

class A(metaclass=MyMeta, debug=True, synchronize=True):
    pass

class B(metaclass=MyMeta):
    pass

class C(metaclass=MyMeta, synchronize=True):
    pass

## \*args, **kwargs에 매개변수 시그니처 강제

In [10]:
from inspect import Signature, Parameter
parms = [Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
         Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
         Parameter('z', Parameter.KEYWORD_ONLY, default=None)]
sig = Signature(parms)
print(sig)

(x, y=42, *, z=None)


In [ ]:
from inspect import Signature, Parameter

def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
             for name in names]
    return Signature(parms)

class Structure:
    __signature__ = make_sig()
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

class Stock(Structure):
    __signature__ = make_sig('name', 'shares', 'price')

class Point(Structure):
    __signature__ = make_sig('x', 'y')

if __name__ == '__main__':
    s1 = Stock('ACME', 100, 490.1)
    print(s1.name, s1.shares, s1.price)

    s2 = Stock(shares=100, name='ACME', price=490.1)
    print(s2.name, s2.shares, s2.price)

    try:
        s3 = Stock('ACME', 100)
    except TypeError as e:
        print(e)

    try:
        s4 = Stock('ACME', 100, 490.1, '6/1/2020')
    except TypeError as e:
        print(e)

    try:
        s5 = Stock('ACME', 100, name='ACME', price=490.1)
    except TypeError as e:
        print(e)

ACME 100 490.1
ACME 100 490.1
missing a required argument: 'price'
too many positional arguments
multiple values for argument 'name'


In [ ]:
from inspect import Signature, Parameter
        
def make_sig(*names):
    parms = [Parameter(name, Parameter.POSITIONAL_OR_KEYWORD)
             for name in names]
    return Signature(parms)

class StructureMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        clsdict['__signature__'] = make_sig(*clsdict.get('_fields',[]))
        return super().__new__(cls, clsname, bases, clsdict)

class Structure(metaclass=StructureMeta):
    _fields = []
    def __init__(self, *args, **kwargs):
        bound_values = self.__signature__.bind(*args, **kwargs)
        for name, value in bound_values.arguments.items():
            setattr(self, name, value)

class Stock(Structure):
    _fields = ['name', 'shares', 'price']

class Point(Structure):
    _fields = ['x', 'y']

if __name__ == '__main__':
    s1 = Stock('ACME', 100, 490.1)
    print(s1.name, s1.shares, s1.price)

    s2 = Stock(shares=100, name='ACME', price=490.1)
    print(s2.name, s2.shares, s2.price)

    try:
        s3 = Stock('ACME', 100)
    except TypeError as e:
        print(e)

    try:
        s4 = Stock('ACME', 100, 490.1, '6/1/2020')
    except TypeError as e:
        print(e)

    try:
        s5 = Stock('ACME', 100, name='ACME', price=490.1)
    except TypeError as e:
        print(e)

ACME 100 490.1
ACME 100 490.1
missing a required argument: 'price'
too many positional arguments
multiple values for argument 'name'


## 클래스 코딩 규칙 강제

In [ ]:
class NoMixedCaseMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError('Bad attribute name: ' + name)
        return super().__new__(cls, clsname, bases, clsdict)

class Root(metaclass=NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self):      # Ok
        pass

print('**** About to generate a TypeError')
class B(Root):
    def fooBar(self):       # TypeError
        pass

**** About to generate a TypeError


TypeError: Bad attribute name: fooBar

* 상속 받았을때 부모의 메소드와 동일하게 파라미터 설정하도록 강제할때
* 메타 클래스는 type을 상속 받아서 정의

In [2]:
from inspect import signature
import logging

class MatchSignaturesMeta(type):
    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)
        sup = super(self, self)
        for name, value in clsdict.items():
            if name.startswith('_') or not callable(value):
                continue
            prev_dfn = getattr(sup,name,None)
            if prev_dfn:
                prev_sig = signature(prev_dfn)
                val_sig = signature(value)
                if prev_sig != val_sig:
                    logging.warning('Signature mismatch in %s. %s != %s',
                                value.__qualname__, str(prev_sig), str(val_sig))

class Root(metaclass=MatchSignaturesMeta):
    pass

class A(Root):
    def foo(self, x, y):
        pass

    def spam(self, x, *, z):
        pass

class B(A):
    def foo(self, a, b):
        pass

    def spam(self,x,z):
        pass

## 클래스를 프로그램적으로 정의

In [ ]:
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price

def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__' : __init__,
    'cost' : cost,
}

import types

Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))

if __name__ == '__main__':
    s = Stock('ACME', 50, 91.1)
    print(s)
    print(s.cost())

4555.0


* named_tuple을 직접 만든 코드
* named_tuple 함수 안에서 classname으로 클래스를 만들어서 cls를 리턴해줌

In [ ]:
import operator
import types
import sys

def named_tuple(classname, fieldnames):
    cls_dict = { name: property(operator.itemgetter(n))
                 for n, name in enumerate(fieldnames) }

    def __new__(cls, *args):
        if len(args) != len(fieldnames):
            raise TypeError('Expected {} arguments'.format(len(fieldnames)))
        return tuple.__new__(cls, (args))

    cls_dict['__new__'] = __new__

    cls = types.new_class(classname, (tuple,), {}, 
                           lambda ns: ns.update(cls_dict))
    cls.__module__ = sys._getframe(1).f_globals['__name__']
    return cls

if __name__ == '__main__':
    Point = named_tuple('Point', ['x', 'y'])
    print(Point)
    p = Point(4, 5)
    print(len(p))
    print(p.x, p[0])
    print(p.y, p[1])
    try:
        p.x = 2
    except AttributeError as e:
        print(e)
    print('%s %s' % p)

<class '__main__.Point'>
2
4 4
5 5
can't set attribute
4 5


## 정의 시 클래스 멤버 초기화

* 위 예제를 type을 상속 받은 클래스로 구현한 버전

In [11]:
import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        print("StructTupleMeta_getter assigned")
        for n, name in enumerate(cls._fields_):
            print(f'\t {n} is {name}')
            setattr(cls, name, property(operator.itemgetter(n)))

class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields_ = []
    def __new__(cls, *args):
        print("Memory Allocated")
        if len(args) != len(cls._fields_):
            raise ValueError('{} arguments required'.format(len(cls._fields_)))
        return super().__new__(cls,args)

class Stock(StructTuple):
    _fields_ = ['name', 'shares', 'price']

class Point(StructTuple):
    _fields_ = ['x', 'y']


if __name__ == '__main__':
    s = Stock('ACME', 50, 91.1)
    print(s)
    print(s[0])
    print(s.name)
    print(s.shares * s.price)
    try:
        s.shares = 23
    except AttributeError as e:
        print(e)


StructTupleMeta_getter assigned
StructTupleMeta_getter assigned
	 0 is name
	 1 is shares
	 2 is price
StructTupleMeta_getter assigned
	 0 is x
	 1 is y
Memory Allocated
('ACME', 50, 91.1)
ACME
ACME
4555.0
can't set attribute
